<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/template_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSE _ GSM

This dataset was too large to be processed in google colab. The gene count and TCC were generated using the following commands:

```
SRA=SRR11181957
GSM=GSM4339772

cd /central/scratch/agalvezm/ace2/counts

/central/home/agalvezm/opt/sratoolkit.2.10.9-centos_linux64/bin/prefetch 2.10.9 ${SRA} --max-size 2000000000

cd ${SRA}

/central/home/agalvezm/opt/sratoolkit.2.10.9-centos_linux64/bin/fasterq-dump --split-files --outdir fastq --include-technical ${SRA}.sra

~/anaconda3/bin/kb ref -d human -i index.idx -g t2g.txt -f1 transcriptome.fasta

~/anaconda3/bin/kb count --h5ad -i index.idx -g t2g.txt -x 10xv2 -o ../results/output${GSM} --filter bustools -t 2 ${SRA}_1.fastq.gz ${SRA}_2.fastq.gz

~/anaconda3/bin/kb count --h5ad -i index.idx -g t2g.txt -x 10xv2 -o ../results/tccoutput${GSM} --tcc --filter bustools -t 2 ${SRA}_1.fastq.gz ${SRA}_2.fastq.gz

```

The unfiltered gene count matrix and TCC produced were uploaded to folders *gc/* and *tcc/* repectively in this run.





In [ ]:
mkdir /content/gc /content/tcc

In [ ]:
# define the values for the analysis

# accession id for the data


database_id = "GSE"
sample_id = "GSM"

# Metadata


database_id = [id] * no_samples

tissue = "blood"

cell_type = "bone marrow"

condition = "ET09"

species = "human"

technology = "10xv2"

paper = "Muus et al 2020"

figure = "Fig 1 a,b  ED Fig 1 a,b,c,d  ED Fig 2 a,b,c,d,e"


env: BAM_LINK=ftp://ftp.sra.ebi.ac.uk/vol1/run/SRR638/SRR6389864/possorted_genome_bam.bam


# Imports and installs

In [ ]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Execute kb ref to get t2g.txt file
!kb ref -d $species -i index.idx -g t2g.txt -f1 transcriptome.fasta


# Load unfiltered matrix and assign filters to each matrix individually

## Load the unfiltered matrices

In [ ]:
# Gunzip matrices
!gunzip /content/gc/adata.h5ad.gz
!gunzip /content/tcc/adata.h5ad.gz

# Load matrices
adata = anndata.read_h5ad("/content/gc/adata.h5ad)
adata_tcc = anndata.read_h5ad("/content/tcc/adata.h5ad)

In [ ]:
# Reformat and fill gene count matrix 

adata = anndata.read_h5ad("output" + sample_id[i] + "/counts_unfiltered/adata.h5ad")
adata.var["gene_id"] = adata.var.index.values

t2g = pd.read_csv("t2g.txt", header=None, names=["tid", "gene_id", "gene_name"], sep="\t")
t2g.index = t2g.gene_id
t2g = t2g.loc[~t2g.index.duplicated(keep='first')]

adata.var["gene_name"] = adata.var.gene_id.map(t2g["gene_name"])
adata.var.index = adata.var["gene_name"]
print("The unfiltered matrix " + sample_id[i] + " contains {} cells by {} genes".format(len(adata.obs), len(adata.var)))

adata.obs["cell_counts"] = adata.X.sum(axis=1)
adata.var["gene_counts"] = nd(adata.X.sum(axis=0))

adata.obs["n_genes"] = nd((adata.X>0).sum(axis=1))
adata.var["n_cells"] = nd((adata.X>0).sum(axis=0))

mito_genes = adata.var_names.str.startswith("MT-" or "mt-") 
adata.obs["percent_mito"] = adata[:,mito_genes].X.sum(axis=1)/adata.X.sum(axis=1)*100

# Changing the name of the index is necessary to write the file (it won't work with duplicated names)
adata.var.index.name = "index"



## Assign filters for gene count matrix

In [2]:
# Filtering criteria
cell_threshold = 100
gene_threshold = 3


In [ ]:
expected_num_cells = 10000#@param {type:"integer"}
knee = np.sort(nd(adata.X.sum(axis=1)))[::-1]

fig, ax = plt.subplots(figsize=(5, 5))

x = knee
y = range(len(knee))

ax.loglog(x, y, linewidth=5, color="g")

ax.axvline(x=knee[expected_num_cells], linewidth=3, color="k")
ax.axhline(y=expected_num_cells, linewidth=3, color="k")

ax.set_xlabel("UMI Counts")
ax.set_ylabel("Set of Barcodes")

plt.show()

cell_threshold = knee[expected_num_cells]

results["cell_threshold" + sample_id[samp_n]] = knee[expected_num_cells]

print ("Cells were filtered down to " + str(expected_num_cells) + " with at least " + str(cell_threshold) + " UMIs")


mito_criteria = 18#@param {type:"integer"}
results["mito_criteria" + sample_id[samp_n]] = mito_criteria

fig, ax = plt.subplots(figsize=(5,5))


x = nd(adata.obs["cell_counts"][adata.obs["cell_counts"] > cell_threshold])
y = nd(adata.obs["percent_mito"][adata.obs["cell_counts"] > cell_threshold])

ax.scatter(x, y, color="green", alpha=0.1)

ax.axhline(y=mito_criteria, linestyle="--", color="k")


ax.set_xlabel("UMI Counts")
ax.set_ylabel("Percent mito")


plt.show()

print("We select " + str(mito_criteria) + " % as the mitochondrial content threshold")


# Filter gene count matrix

In [ ]:
adata.obs["pass_count_filter"] = adata.obs["cell_counts"] > results["cell_threshold" + sample_id[i]]
adata.obs["pass_mito_filter"] = adata.obs.percent_mito < results["mito_criteria" + sample_id[i]]
adata.var["pass_gene_filter"] = adata.var["n_cells"] > gene_threshold

cell_mask = np.logical_and(adata.obs["pass_count_filter"].values, adata.obs["pass_mito_filter"].values)
gene_mask = adata.var["pass_gene_filter"].values

print("Current Shape: {:,} cells x {:,} genes".format(adata.shape[0], adata.shape[1]))
print("    New shape: {:,} cells x {:,} genes".format(cell_mask.sum(), gene_mask.sum()))
results["data_" + sample_id[i]] = adata[cell_mask, gene_mask]


# Anotate and write gene count matrix

In [ ]:
adata.uns["database_id"] = database_id

adata.uns["tissue"] = tissue

adata.uns["cell_type"] = cell_type

adata.uns["sample_id"] = sample_id

adata.uns["condition"] = condition

adata.uns["species"] = species

adata.uns["technology"] = technology

adata.uns["paper"] = paper

adata.uns["figure"] = figure

%cd /content

adata.write("result" + sample_id)

Filter TCC using gene count matrix's filter

In [ ]:
adata_tcc = adata_tcc[adata.obs.index.values]
# transfer obs data
adata_tcc.obs = adata.obs
# transfer metadata
adata_tcc.uns = adata.uns
# write tcc matrix
adata_tcc.write("/content/tcc_" + sample_id)
#gzip tcc matrix
cmd = "gzip /content/tcc_" + sample
!$cmd